# ETL_Project

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

### Store SARS csv files into dataframe

In [ ]:
sars_file = "Resources/sars_2003_complete_dataset_clean.csv"
df_sars = pd.read_csv(sars_file)
df_sars.head()

### Transform SARS data

In [ ]:
df_sars = df_sars[['Country', 'Cumulative number of case(s)', 'Number of deaths']].copy()
df_sars.columns = ['country', 'total_cases_sars', 'total_deaths_sars']
df_sars = df_sars.groupby('country').max().reset_index()
df_sars.head()

In [ ]:
df_sars.head()

### Store nCoV csv files into dataframe

In [ ]:
ncov_file = "Resources/2019_nCoV_data.csv"
df_ncov = pd.read_csv(ncov_file)
df_ncov.head()

### Transform SARS data

In [ ]:
# Rename cities in China.
df_ncov = df_ncov.replace(to_replace='China', value='Mainland China', regex=False)
df_ncov = df_ncov.replace(to_replace='Hong Kong', value='Hong Kong SAR, China', regex=False)
df_ncov = df_ncov.replace(to_replace='Macau', value='Macau SAR, China', regex=False)
df_ncov = df_ncov.replace(to_replace='Taiwan', value='Taiwan, China', regex=False)

In [ ]:
df_ncov[['Confirmed', 'Deaths']] = df_ncov[['Confirmed', 'Deaths']].astype(int)
df_ncov = df_ncov.groupby('Country')['Confirmed', 'Deaths'].max().reset_index()
df_ncov.columns = ['country', 'total_cases_ncov', 'total_deaths_ncov']

In [ ]:
df_ncov.head()

### Connect to local database

In [ ]:
rds_connection_string = "postgres:postgres@localhost:5432/virus_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

### Load

In [ ]:
df_sars.to_sql(name='sars', con=engine, if_exists='append', index=False)
df_ncov.to_sql(name='ncov', con=engine, if_exists='append', index=False)

# Tyler Code, Ebola Data

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from datetime import datetime

Import Ebola Data

In [ ]:
ebola_file = "Resources/ebola_data_db_format.csv"
ebola = pd.read_csv(ebola_file)
ebola

Drop Rows with 0/NaN Values

In [ ]:
ebola_df = ebola[ebola != 0].dropna()

Seperate into 2 seperate DF by Indicator String Value .\
(Confirmed Cases, Confirmed Deaths)

In [ ]:
e_confirm = ebola_df.loc[ebola_df['Indicator'] == "Cumulative number of confirmed Ebola cases"]
e_confirm.rename(columns = {"value": "Confirmed Cases to Date"}, inplace = True)
e_confirm.drop(columns = ['Indicator'], inplace = True)
e_confirm
#print(e_confirm.to_string())

In [ ]:
e_death = ebola_df.loc[ebola_df['Indicator'] == "Cumulative number of confirmed Ebola deaths"]
e_death.rename(columns = {"value": "Deaths to Date"}, inplace = True)
e_death.drop(columns = ['Indicator'], inplace = True)
e_death
#print(e_death.to_string())

Merge the Seperated Confirmed Cases to Date and Deaths to Date DF's .\ 
Rename Columns .\
Fill NaN with 0 .\
Confirm Cases and Deaths as int Type .\
Change date to datetime

In [ ]:
ebola_complete = e_confirm.merge(e_death, how = 'outer', left_index = False, right_index = False)
ebola_complete.rename(columns = {"Country": "country", "Date": "date", "Confirmed Cases to Date": "total_cases_ebola", "Deaths to Date": "total_deaths_ebola"}, inplace = True)
ebola_complete['total_deaths_ebola'] = ebola_complete['total_deaths_ebola'].fillna(0)
ebola_complete = ebola_complete.astype({"total_cases_ebola": int, "total_deaths_ebola": int})
pd.to_datetime(ebola_complete['date'])
ebola_complete
#print(ebola_complete.to_string())

### Tyler; Database Connection and Upload

In [ ]:
ebola_connection_string = "postgres:postgres@localhost:5432/virus_db"
engine = create_engine(f'postgresql://{ebola_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
ebola_complete.to_sql(name = 'ebola', con = engine, if_exists = 'append', index = False)